In [5]:
import gurobipy as gp
from gurobipy import GRB

# 데이터 입력
material_names = ["원두", "우유", "탄산수", "유자청", "자몽청"]
u_i = [1000,6000,6000,2000,1500]
P_i = [9900,12000,8890,9600,23500]
ss_i = [4630,16835,1548,374,232]
DM_kj = [
    [128, 145, 105, 114, 125, 98, 98],
    [85, 96, 69, 75, 82, 64, 64],
    [14, 16, 11, 12, 13, 10, 10],
    [9, 10, 7, 8, 8, 7, 7],
    [7, 8, 5, 6, 6, 5, 5]
]
D_ij = [[5121, 5801,4181,4537,4991,3889,3889], 
        [18619,21094,15202,16498,18148, 14141,14141],
        [1712,1939,1398,1517,1668,1300,1300], 
        [414,469,338,367,403,314,314], 
        [257,291,210,228,250,195,195]
]
S_k = [2000, 2500, 3000, 3300, 2500]
I_i0 = [5130,18620,1720,420,260]
M = 100000000
Fixed_cost = 1713423
Capa = 62000


# Gurobi 모델 생성
model = gp.Model("Inventory Management")

# 변수 정의
b = model.addVars(5, 7, vtype=GRB.BINARY, name="b")
Q = model.addVars(5, 7, vtype=GRB.INTEGER, name="Q")
I = model.addVars(5, 8, vtype=GRB.CONTINUOUS, name="I")

# 초기 재고 설정
model.addConstr(I[0, 0] == I_i0[0])
model.addConstr(I[1, 0] == I_i0[1])
model.addConstr(I[2, 0] == I_i0[2])
model.addConstr(I[3, 0] == I_i0[3])
model.addConstr(I[4, 0] == I_i0[4])


# 제약 조건 설정
for i in range(5):
    for j in range(7):
        model.addConstr(Q[i, j] <= b[i, j] * M)
        model.addConstr(I[i, j + 1] == I[i, j] + u_i[i] * Q[i, j] - D_ij[i][j])
        model.addConstr(I[i, j + 1] >= ss_i[i])
    model.addConstr(gp.quicksum(b[i, j] for j in range(7)) <= 3)

for j in range(1, 8):
    model.addConstr(gp.quicksum(I[i, j] for i in range(5)) <= 62000)

# 목적 함수 설정
revenue = gp.quicksum(DM_kj[k][j] * S_k[k] for k in range(5) for j in range(7))
cost = gp.quicksum(Q[i, j] * P_i[i] for i in range(5) for j in range(7)) + gp.quicksum(3000 * b[i, j] for i in range(5) for j in range(7)) + Fixed_cost

model.setObjective(revenue - cost, GRB.MAXIMIZE)

# 최적화 수행
model.optimize()

# 결과 출력
if model.status == GRB.OPTIMAL:
    for i in range(5):
        for j in range(7):
            print(f"Day {j + 1}, {material_names[i]}: Order {Q[i, j].x} units, Inventory {I[i, j + 1].x} units, Order placed: {b[i, j].x}")
else:
    print("Optimal solution not found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 122 rows, 110 columns and 285 nonzeros
Model fingerprint: 0x425111a3
Variable types: 40 continuous, 70 integer (35 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [3e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+04]
Presolve removed 119 rows and 105 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 7 nonzeros
Variable types: 0 continuous, 5 integer (2 binary)
Found heuristic solution: objective 1121097.0000

Root relaxation: objective 1.123954e+06, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [4]:
import gurobipy as gp
from gurobipy import GRB

# 데이터 입력
material_names = ["원두", "우유", "탄산수", "유자청", "자몽청"]
u_i = [1000, 6000, 6000, 2000, 1500]
P_i = [9900, 12000, 8890, 9600, 23500]
ss_i = [4630, 16835, 1548, 374, 232]
DM_kj = [
    [128, 145, 105, 114, 125, 98, 98],
    [85, 96, 69, 75, 82, 64, 64],
    [14, 16, 11, 12, 13, 10, 10],
    [9, 10, 7, 8, 8, 7, 7],
    [7, 8, 5, 6, 6, 5, 5]
]
D_ij = [
    [5121, 5801, 4181, 4537, 4991, 3889, 3889],
    [18619, 21094, 15202, 16498, 18148, 14141, 14141],
    [1712, 1939, 1398, 1517, 1668, 1300, 1300],
    [414, 469, 338, 367, 403, 314, 314],
    [257, 291, 210, 228, 250, 195, 195]
]
S_k = [2000, 2500, 3000, 3300, 2500]
I_i0 = [5130, 18620, 1720, 420, 260]
M = 100000000
Fixed_cost = 1713423
Capa = 62000

# Gurobi 모델 생성
model = gp.Model("Inventory Management")

# 변수 정의
b = model.addVars(5, 7, vtype=GRB.BINARY, name="b")
Q = model.addVars(5, 7, vtype=GRB.INTEGER, name="Q")
I = model.addVars(5, 8, vtype=GRB.CONTINUOUS, name="I")
e = model.addVars(7, vtype=GRB.BINARY, name="e")

# 초기 재고 설정
for i in range(5):
    model.addConstr(I[i, 0] == I_i0[i])

# 제약 조건 설정
for i in range(5):
    for j in range(7):
        if i == 0:
            model.addConstr(I[i, j + 1] == I[i, j] + u_i[i] * Q[i, j] - D_ij[i][j] * (1 / 3 + 2 * (1 + e[j] * 1.28) / 3))
        else:
            model.addConstr(I[i, j + 1] == I[i, j] + u_i[i] * Q[i, j] - D_ij[i][j])
        model.addConstr(Q[i, j] <= b[i, j] * M)
        model.addConstr(I[i, j + 1] >= ss_i[i])
    model.addConstr(gp.quicksum(b[i, j] for j in range(7)) <= 3)

for j in range(1, 8):
    model.addConstr(gp.quicksum(I[i, j] for i in range(5)) <= Capa)

# 이벤트는 하루만 가능
model.addConstr(gp.quicksum(e[j] for j in range(7)) == 1)

# 목적 함수 설정
revenue = (gp.quicksum(DM_kj[k][j] * S_k[k] for k in range(1, 5) for j in range(7)) +
           gp.quicksum(DM_kj[0][j] * (1 + e[j] * 1.28) * S_k[0] * (1 - e[j] * 0.5) for j in range(7)))
cost = (gp.quicksum(Q[i, j] * P_i[i] for i in range(5) for j in range(7)) +
        gp.quicksum(3000 * b[i, j] for i in range(5) for j in range(7)) +
        Fixed_cost)

model.setObjective(revenue - cost, GRB.MAXIMIZE)

# 최적화 수행
model.optimize()

# 결과 출력
if model.status == GRB.OPTIMAL:
    for i in range(5):
        for j in range(7):
            print(f"Day {j + 1}, {material_names[i]}: Order {Q[i, j].x} units, Inventory {I[i, j + 1].x} units, Order placed: {b[i, j].x}")
    for j in range(7):
        print(f"Day {j + 1}: Event held: {e[j].x}")
else:
    print("Optimal solution not found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 123 rows, 117 columns and 299 nonzeros
Model fingerprint: 0x4b400694
Model has 7 quadratic objective terms
Variable types: 40 continuous, 77 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [3e+03, 2e+05]
  QObjective range [3e+05, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+04]
Presolve removed 104 rows and 91 columns
Presolve time: 0.00s
Presolved: 19 rows, 26 columns, 57 nonzeros
Variable types: 0 continuous, 26 integer (12 binary)
Found heuristic solution: objective 1108937.0000

Root relaxation: objective 1.119583e+06, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     